In [5]:
import requests
from requests_oauthlib import OAuth1
import os, json
from time import sleep


url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1('E5eeHHPrKm0N8qkHhH1BOkSu6', 'iLkXetVRI93Xt32KJfY44CUfau1BgH4BHSotaDLHloaykLOmTR',
                  '744995423308374017-ic5GQxFdUxYjgpFJSbAQ1rRaJluz4nJ', 'hZl4AAiu62CalzpY8BeJfX2wJXezROk0WsEhms9KxXQ4w')
requests.get(url, auth=auth)


searchWord = input("Please enter a search term: ")
print("This is the search term: " + searchWord)
searchWord = searchWord.replace(" ", "%20")

Please enter a search term: acmilan
This is the search term: acmilan


In [6]:


getTweet = 'https://api.twitter.com/1.1/search/tweets.json?q=%40'+searchWord+'&count=100'
#print(requests.get(getTweet, auth=auth))
#print("URL: " + getTweet)
#response = requests.get(getTweet, auth=auth)


#print(data)
#json_str = json.dumps(data)
#data = json.loads(json_str)
# Writing JSON data


def merge(query):
    data = {}
    while len(data) < 500:
        response = requests.get(getTweet, auth = auth) 
        response = response.json()
        for status in response['statuses']:
            data[status['id']] = status 
        sleep(2)
        print(len(data))
    return data

data = merge(getTweet)
with open('data.json', 'w') as f:
     json.dump(data, f)
        


100
103
109
112
120
123
123
125
128
132
133
135
139
140
143
146
149
156
160
162
172
176
177
178
179
181
181
182
186
193
198
199
202
202
206
209
214
216
222
230
231
234
235
238
245
245
246
252
256
261
265
266
269
276
280
286
290
292
294
298
303
305
310
314
320
322
325
327
334
336
344
351
358
367
376
384
387
397
407
416
425
436
443
449
456
459
462
468
478
484
493
503


In [7]:
print(len(data))

503


In [12]:
#TimeZones
with open('data.json', 'r') as fr:
    data = json.load(fr)
time_zone_all = {}
for id in data:
    user = data[id]['user']
    if user['time_zone'] is not None: 
        if user['time_zone'] in time_zone_all:
            time_zone_all[user['time_zone']] += 1
        else:
            time_zone_all[user['time_zone']] = 1

print(time_zone_all)
print(sorted(time_zone_all, key=lambda x: time_zone_all[x]))

top10 = sorted(time_zone_all, key=lambda x: time_zone_all[x])
print(top10[:10])

{'Buenos Aires': 2, 'Stockholm': 2, 'Jakarta': 38, 'Bogota': 3, 'Central Time (US & Canada)': 18, 'Alaska': 1, 'Nairobi': 1, 'America/Caracas': 1, 'Greenland': 6, 'Melbourne': 1, 'Arizona': 3, 'Riyadh': 12, 'Ljubljana': 1, 'Europe/Moscow': 1, 'Mountain Time (US & Canada)': 1, 'Brasilia': 4, 'Cairo': 2, 'Skopje': 1, 'Bangkok': 20, 'Baghdad': 5, 'Indiana (East)': 1, 'Mexico City': 4, 'Madrid': 2, 'Warsaw': 1, 'Atlantic Time (Canada)': 2, 'Dublin': 2, 'Paris': 3, 'London': 13, 'Muscat': 6, 'West Central Africa': 5, 'Tokyo': 3, 'Lima': 1, 'Beijing': 3, 'Casablanca': 2, 'Caracas': 7, 'Pacific Time (US & Canada)': 38, 'Quito': 1, 'Rome': 23, 'Mid-Atlantic': 1, 'Amsterdam': 9, 'Bern': 1, 'Belgrade': 3, 'Kyiv': 1, 'Santiago': 1, 'Sydney': 2, 'Central America': 6, 'Athens': 13, 'Hawaii': 11, 'Eastern Time (US & Canada)': 12, 'Kuala Lumpur': 6, 'Bucharest': 1}
['Alaska', 'Nairobi', 'America/Caracas', 'Melbourne', 'Ljubljana', 'Europe/Moscow', 'Mountain Time (US & Canada)', 'Skopje', 'Indiana (Ea

In [71]:
#emoticons
emoticons = []
for id in data:
    text = data[id]['text']
    emoticons.append(text.split())    

#rint(emoticons)

list_of_codes = ['U+1F601','U+1F602','U+1F603','U+1F604','U+1F605','U+1F606','U+1F607','U+1F6018','U+1F609','U+1F60A'] 
matching = []
for emo in emoticons:
    matching += [token for token in emo if token.startswith('\U000')]
    
print(matching)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-4: truncated \UXXXXXXXX escape (<ipython-input-71-61b834bbc05c>, line 12)

In [33]:
print(t.tokenize("😀"))

['😀']


In [43]:
print(u"\U0001F600")

😀
